# S2.02 - Notebook n°2

## Partie 2 - Recherche de plus court chemin

### BERHO Andoni
### BOURCIEZ Maxime
### TD II - TP 3

#### Algorithmes de recherche des plus courts chemins

# I - Importation des données

In [13]:
import json
import pandas as pd
import numpy as np
import os
import time


# os.chdir("C:\\IUT\\Semestre 2\\S2.02 - Explo algorithmique d'un problème\\partie2") # Pc portable Maxime
os.chdir("E:\\Cours\\Semestre2\\S2.02\\S2.02-Explo.-d-un-pb.-algo\\partie2") # PC Home Maxime

# import dicsucc.json et dicsuccdist.json (--> dictionnaire)
with open("dicsucc.json", "r") as fichier:
    dicsucc = json.load(fichier)
with open("dicsuccdist.json", "r") as fichier:
    dicsuccdist = json.load(fichier)

# import aretes.csv (--> dataframe) et transformation de lstpoints (chaîne-->liste)
aretes = pd.read_table('aretes.csv', sep  =';', index_col= 0)

for ind in aretes.index :
    ls = aretes.loc[ind,'lstpoints'].replace(" ","").replace("]", "").replace("[", "").split(',')
    lst = []
    for val in ls :
        lst.append(int(val))
    aretes.at[ind,'lstpoints'] = lst


# import sommets.csv, matrice_poids.csv (--> dataframe)
sommets = pd.read_table('sommets.csv', sep  =';', index_col= 0)
matrice_poids = pd.read_csv('matrice_poids.csv', sep = ';', index_col = 0)
sommets['indice'] = [i for i in range(len(sommets.index))]

# transformation dataframe matrice des poids en tableau    
tableau_poids = np.array(matrice_poids)

# transformation matrice des poids en liste de listes
liste_poids = [[None for j in range(len(tableau_poids))] for i in range(len(tableau_poids))]
for i in range(len(tableau_poids)):
    for j in range(len(tableau_poids)):
        liste_poids[i][j]  = tableau_poids[i,j]


del fichier, i, j, val, ls, lst, ind 

# II - Transformation du graphe

In [2]:
def transformer_graphe(graphe):
    """ Le graphe d'origine incluait des clés en string, et nous préférons par simplicité les transformer en entier.
        De plus, les valeurs du dictionnaire d'origine étaient des listes de listes, et nous préférons, pour manipuler, des dictionnaires.
        Cette fonction transforme le dictionnaire dans la forme que nous le voulons"""
    nouveau_graphe = {}
    # On itère sur les sommet (et leurs successeurs)
    for sommet_str, voisins in graphe.items():
        sommet_int = int(sommet_str)     # Transformation de la clé
        nouveau_graphe[sommet_int] = {}  # Création du couple clé-valeur dans le nouveau dictionnaire
        for voisin, poids in voisins:
            nouveau_graphe[sommet_int][voisin] = poids  # Ajout dans le dictionnaire du voisins le poid de l'arc
    return nouveau_graphe

graphe_transforme = transformer_graphe(dicsuccdist)

# III - Fonction de reconstitution du chemin

In [3]:
def reconstituer(pred, dep, arr):
    chemin = []
    sommet = arr
    while sommet != dep:
        chemin.insert(0, sommet)
        sommet = pred[sommet]
    chemin.insert(0, dep)

    return chemin

# IV - Algorithme de dijkstra

In [12]:


def dijkstra(graphe, depart, arrivee):
    

    
    # Initialisation
    distances = {sommet: float('inf') for sommet in graphe}
    distances[depart] = 0
    predecesseurs = {}
    non_traites = set(graphe.keys())

    while non_traites:
        # Sélectionner le sommet non traité avec la plus petite distance
        sommet_courant = min(non_traites, key=lambda sommet: distances[sommet])
        non_traites.remove(sommet_courant)

        if sommet_courant == arrivee:
            break  # On a trouvé le chemin le plus court

        for voisin, poids in graphe[sommet_courant].items():  # On itère sur les voisins
            # Calculer la nouvelle distance
            nouvelle_distance = distances[sommet_courant] + poids

            # Vérifier si la nouvele distance est meilleure
            if nouvelle_distance < distances[voisin]:
                distances[voisin] = nouvelle_distance
                predecesseurs[voisin] = sommet_courant

    # Reconstruction du chemin le plus court
    return reconstituer(predecesseurs, depart, arrivee)

    
# Lancement du chrono
startDijkstra = time.time()

cheminDijkstra = dijkstra(graphe_transforme, 1806175538, 1801848709)

endDijkstra = time.time()
print("Temps d'exécution = ", endDijkstra - startDijkstra) 

Temps d'exécution =  0.23988604545593262


# V - Algorithme de Belmann

In [14]:
def bellman_ford(graphe, depart, arrivee):
    # Initialisation
    distances = {sommet: float('inf') for sommet in graphe}
    distances[depart] = 0
    predecesseurs = {}

    # Nombre de sommets dans le graphe
    nb_sommets = len(graphe)

    # Relaxation des arêtes
    for _ in range(nb_sommets - 1):
        for sommet in graphe:
            for voisin, poids in graphe[sommet].items():
                if distances[sommet] + poids < distances[voisin]:
                    distances[voisin] = distances[sommet] + poids
                    predecesseurs[voisin] = sommet

    # Détection de cycle négatif
    for sommet in graphe:
        for voisin, poids in graphe[sommet].items():
            if distances[sommet] + poids < distances[voisin]:
                return "Cycle négatif détecté"

    # Reconstruction du chemin le plus court
    return reconstituer(predecesseurs, depart, arrivee)

# Lancement du chrono
startDijkstra = time.time()

cheminBellmanFord = bellman_ford(graphe_transforme, 1806175538, 1801848709)

endDijkstra = time.time()
print("Temps d'exécution = ", endDijkstra - startDijkstra) 

Temps d'exécution =  2.5448720455169678


# VI - Algorithme de Floyd-Warshall

In [ ]:
def floyd_warshall(matricePonderee):
    taille = len(matricePonderee)
    
    # Remplissage de M0 et P0
    M = np.array(matricePonderee)
    P = np.full((taille, taille), -1, dtype=int)
    
    for i in range(taille):
        for j in range(taille):
            if M[i][j] != 0 and i != j:
                P[i][j] = i
            else:
                P[i][j] = -1  # Utilisation de -1 pour indiquer aucun prédécesseur
    start = time.time()
    
    # Début des itérations des lignes et colonnes
    for k in range(taille):
        for i in range(taille):
            for j in range(i, taille):
                # Relâchement
                if M[i][k] + M[k][j] < M[i][j]:
                    M[i][j] = M[i][k] + M[k][j]
                    M[j][i] = M[i][k] + M[k][j]
                    P[i][j] = P[k][j]
        currentTime = time.time()
        
        print ("étape ", k, " terminée || temps : ", currentTime - start)
    
    return M, P
                        
(M, P) = floyd_warshall(matrice_poids)
                
# Sauvegarder M dans un fichier CSV
np.savetxt("M_Floyd_Warshall.csv", M, delimiter=",", fmt="%s")

# Sauvegarder P dans un fichier CSV
np.savetxt("P-Floyd_Warshall.csv", P, delimiter=",", fmt="%d")

## B) Exécutions possibles

In [ ]:
# Importation des données
M = pd.read_table('M_Floyd_Warshall.csv', sep=',', decimal='.')
P = pd.read_table('P-Floyd_Warshall.csv', sep=',', decimal='.')

# Appel de la fonction de reconstition
def reconstituer_chemin(P, depart, arrivee):
    # convertir les sommets avec leurs index
    depart = sommets.iloc['id_point'][depart]
    
    chemin = []
    noeud = arrivee
    while noeud != depart:
        chemin.insert(0, noeud)
        noeud = P[depart][noeud]
        if noeud == -1:
            return None  # Aucun chemin trouvé
    chemin.insert(0, depart)
    if chemin:
        print("Chemin trouvé :", chemin)
    else:
        print("Aucun chemin trouvé entre", depart, "et", arrivee)
    return chemin

chemin = reconstituer_chemin(P,1806175538, 1801848709)



# VII - Algorithme A*

In [ ]:
import heapq

def a_star(graphe, depart, arrivee):
    # Calculer les distances à vol d'oiseau entre chaque sommet et l'arrivée
    distances_estimees = {sommet: distanceGPS(sommets.loc[sommet, 'lat'], sommets.loc[arrivee, 'lat'], sommets.loc[sommet, 'lon'], sommets.loc[arrivee, 'lon']) for sommet in graphe}
    
    # Initialisation
    ouverts = [(distances_estimees[depart], depart)]  # (estimation + distance réelle, sommet)
    fermes = set()
    predecesseurs = {}
    distances = {sommet: float('inf') for sommet in graphe}
    distances[depart] = 0
    
    while ouverts:
        _, sommet_courant = heapq.heappop(ouverts)
        
        if sommet_courant == arrivee:
            # Reconstruction du chemin le plus court
            chemin = []
            while sommet_courant != depart:
                chemin.append(sommet_courant)
                sommet_courant = predecesseurs[sommet_courant]
            chemin.append(depart)
            return chemin[::-1]
        
        fermes.add(sommet_courant)
        
        for voisin, poids in graphe[sommet_courant].items():
            if voisin in fermes:
                continue
            
            nouvelle_distance = distances[sommet_courant] + poids
            if nouvelle_distance < distances[voisin]:
                distances[voisin] = nouvelle_distance
                heapq.heappush(ouverts, (nouvelle_distance + distances_estimees[voisin], voisin))
                predecesseurs[voisin] = sommet_courant
    
    return None  # Pas de chemin trouvé